In [1]:
import pandas as pd
import numpy as np
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score



In [2]:
df_temp_out = pd.read_csv('outdoor_temperature.csv')  
df_temp_in = pd.read_csv('indoor_temperature.csv')  
df_heat = pd.read_csv('heating_consumption.csv')  

df_combined = pd.concat([df_temp_out, df_temp_in, df_heat], axis=1)  
date_range = pd.date_range(start='2021-01-01', end='2022-12-28', freq='D')
df_combined['date'] = date_range
print(df_combined)

     5.2000  4.2000  4.4000  3.3000  1.7000  2.8000  1.9000  1.9000.1  2.0000  \
0       0.9     1.4     1.6     0.8    -0.4    -1.1    -1.2      -0.7    -0.7   
1       0.8     0.9     0.9     1.1     1.3     1.3     0.1      -0.6     1.9   
2       0.5     0.2     0.1     0.6     0.3    -0.4    -0.4       0.1     0.8   
3       1.9     1.7     1.3     1.5     1.5     1.3     1.0       0.9     1.1   
4       1.4     1.5     1.5     1.5     1.4     1.5     1.5       1.4     1.7   
..      ...     ...     ...     ...     ...     ...     ...       ...     ...   
722     7.1     7.2     7.6    10.5    11.7     8.5     7.7       8.0    11.0   
723     8.1     7.8     6.9     7.0     5.8     7.8     6.5       7.5     9.1   
724    10.9    11.0    10.7    10.9    10.9    10.6    10.5      10.7    11.0   
725     6.5     6.9     6.3     5.8     4.9     4.8     4.0       5.1     7.3   
726     9.3     8.9     8.7     8.7     9.2     9.1    10.3       9.1     9.8   

     2.3000  ...  0.0000.15

In [3]:
df_no_heating = df_combined[df_combined.iloc[:, -25:-1].sum(axis=1) == 0]  
df_heating = df_combined[df_combined.iloc[:, -25:-1].sum(axis=1) != 0]      

#print(df_no_heating)
df_heating = df_heating.reset_index(drop=True)

print(df_heating)



     5.2000  4.2000  4.4000  3.3000  1.7000  2.8000  1.9000  1.9000.1  2.0000  \
0       0.5     0.2     0.1     0.6     0.3    -0.4    -0.4       0.1     0.8   
1       1.9     1.7     1.3     1.5     1.5     1.3     1.0       0.9     1.1   
2       1.4     1.5     1.5     1.5     1.4     1.5     1.5       1.4     1.7   
3      -3.0    -3.0    -3.4    -3.7    -4.7    -4.6    -4.6      -4.6    -3.0   
4      -3.6    -4.2    -3.8    -4.8    -4.8    -5.0    -5.1      -4.9    -2.4   
..      ...     ...     ...     ...     ...     ...     ...       ...     ...   
240    12.1    12.1    12.3    12.3    12.4    12.2    12.3      12.4    12.8   
241    12.6    12.2    12.0    12.6    12.7    12.6    12.9      13.1    13.3   
242    12.3    11.9    11.9    11.1    10.8    10.5    10.3      10.5    11.0   
243    10.8    10.6    11.1     9.3     9.0     7.9     8.6       9.0    10.6   
244    12.2    10.8    10.6    10.2     9.5     9.3     8.9       8.9    10.8   

     2.3000  ...  0.0000.15

In [4]:
heating_indices = df_heating.index
#print(heating_indices)
df_temp_out_heating = df_temp_out.iloc[heating_indices]
df_temp_in_heating = df_temp_in.iloc[heating_indices]
df_heat_heating = df_heat.iloc[heating_indices]

#print(df_temp_out_heating)
#print(df_temp_in_heating)
#print(df_heat_heating)

In [5]:
def normalize_data(data):
    scaler = TimeSeriesScalerMeanVariance()
    data_normalized = scaler.fit_transform(data.values.reshape((data.shape[0], data.shape[1], 1)))
    return pd.DataFrame(data_normalized.reshape(data_normalized.shape[0], data_normalized.shape[1]), columns=data.columns)


In [6]:

def apply_kmeans(data,data_normalized, n_clusters=3):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    data_normalized['cluster'] = kmeans.fit_predict(data_normalized)
    data['cluster']=data_normalized['cluster']
    return kmeans, data
